In [2]:
from sklearn.externals import joblib

In [9]:
## load test model
clf1 = joblib.load('classify_test1.model') # 분류기 엔진
cate_dict1 = joblib.load('cate_dict1.dat') # id별 카테고리가 맵핑되어있는 파일
vectorizer1 = joblib.load('vectorizer_test1.dat') # 제목들을 숫자 id로 변환해주는 것
clf2 = joblib.load('classify_test2.model') 
cate_dict2 = joblib.load('cate_dict2.dat') 
vectorizer2 = joblib.load('vectorizer_test2.dat') 
clf = joblib.load('classify_test.model') # 분류기 엔진
cate_dict = joblib.load('cate_dict.dat') # id별 카테고리가 맵핑되어있는 파일
vectorizer = joblib.load('vectorizer_test.dat') # 제목들을 숫자 id로 변환해주는 것

In [10]:
cate_id_name_dict1 = dict(map(lambda (k,v):(v,k),cate_dict1.items()))
cate_id_name_dict2 = dict(map(lambda (k,v):(v,k),cate_dict2.items()))

In [11]:
# initialize size
size = 0
joblib.dump(size,'size.dat')

['size.dat']

In [12]:
# define removeStopWords

mapping = { 'A/S':'AS', u'현대백화점':u'백화점', u'롯데백화점':u'백화점', u'대구백화점':u'백화점', 'IntelIntel':'Intel', 'IBMIBM':'IBM'}
stop_words = [u"%즉시할인쿠폰",  u"5% 즉시할인", u"0% 즉시할인", u"% 즉시할인", u"5% 할인", u"0% 할인", u"5%할인", u"0%할인", u"% 할인", u"적립",
              u"쿠폰", u"즉시",u"정품", u"증정", u"할인", u"하나 더", u"하나더", u"사은품", u"중고",
              u"해외직배송", u"직배송", u"무료배송", u"안전배송", u"당일", u"배송", u"발송", u"빠른", u"저렴한", u"새로운",
              u"신한카드", u"BC카드", u"KB국민카드", u"국민카드", u"롯데카드", u"KB카드", u"현대카드", u"삼성카드", 
              u"비씨카드", u"우리카드", u"제휴카드", u"하나카드", u"씨티카드",
              u"일까지"]
symbols = ["[","]","(",")", "+", "-", "/", "?", "~", "*", u"☆", u"★", u"●", u"♥", u"■", u"▶", u"◀", u"↑"]
ambg_words = [u"해외", u"구매대행"]

start = 0
length = 1
remove_ambg = True

# define concatenate function
def concate(list1, list2) :
    for each in list2:
        list1.append(each)
    return list1

remove_list = concate(stop_words, symbols)
if remove_ambg:
    remove_list = concate(remove_list,ambg_words)
    
def removeStopWords(word) :
     # replace
    for before, after in mapping.iteritems():
        word = word.replace(before, after)
    #remove
    for each in remove_list :
        word = word.replace(each," ")
    return word

In [13]:
# define splitMorph using konlpy

from konlpy.tag import Twitter
from konlpy.utils import pprint
twit = Twitter()

def splitMorph(word) :
    morphs = twit.pos(word)
    
    words = ""
    for morph in morphs :
        word = morph[0]
        words += " " + word
    use_nouns = True
    if use_nouns :
        nouns = twit.nouns(word)
        for noun in nouns :
            words += " " + noun
    return words

In [14]:
# read json results of samples
list_caffe_samples = joblib.load('list_caffe_samples.dat')
list_caffe_test = joblib.load('list_caffe_test.dat')

is_test = False

if is_test :
    list_caffe = list_caffe_test
else :
    list_caffe = list_caffe_samples
print len(list_caffe)

206


In [16]:
# server for 206 samples tests

from bottle import route, run, template,request,get, post
import  time
from threading import  Condition
_CONDITION = Condition()

# initialize size
size = 0
joblib.dump(size,'size.dat')

@route('/classify')
def classify():
    print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '').decode('utf-8')
    
    name = removeStopWords(name)
    name = splitMorph(name)
    
    # read size
    size = joblib.load('size.dat')
    
    # append caffe result
    res = list_caffe[size]['result']
    
    if res[0] :
        for each in res[2] :
            name += " " + each[0].replace(" ", "") + " " + each[0].replace(" ", "")
            if float(each[1]) > 0.7 :
                name += " " + each[0]
            if float(each[1]) > 1.0 :
                name += " " + each[0]
            
    
    pred = clf.predict(vectorizer.transform([name]))[0]
    if pred == 1 :
        pred1 = clf1.predict(vectorizer1.transform([name]))[0]
        cate = cate_id_name_dict1[pred1]
    else :
        pred2 = clf2.predict(vectorizer2.transform([name]))[0]
        cate = cate_id_name_dict1[pred2]
        
    # update and store size
    if size % 100 == 0:
        print "%d saved" % size
        print name
        print cate
    
    size = size + 1;
    joblib.dump(size,'size.dat')
    
    return {'cate':cate}


run(host='0.0.0.0', port=8887)
#  http://somaeval.hoot.co.kr:8884/eval?url=http://  :8887&mode=all&name=oyj

Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



classify called
0 saved
 남성 남성 website website web site web site computer computer computer computer machine machine machine machine device device container container
디지털/가전;저장장치;HDD


192.168.99.1 - - [24/Aug/2016 12:02:42] "GET /classify?name=%EB%82%A8%EC%84%B1&img=jkh HTTP/1.1" 200 72


classify called


192.168.99.1 - - [24/Aug/2016 12:03:08] "GET /classify?name=%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EC%9D%98%EB%A5%98%20%EB%82%A8%EC%84%B1%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EC%9D%98%EB%A5%98%20%EB%82%A8%EC%84%B1&img=jkh HTTP/1.1" 200 89


classify called


192.168.99.1 - - [24/Aug/2016 12:03:34] "GET /classify?name=%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EC%9D%98%EB%A5%98%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EC%9D%98%EB%A5%98%20%EB%82%A8%EC%84%B1&img=jkh HTTP/1.1" 200 89


classify called


192.168.99.1 - - [24/Aug/2016 12:03:44] "GET /classify?name=%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EC%9D%98%EB%A5%98%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EC%9E%AC%ED%82%B7%20%EC%9E%AC%ED%82%B7%20%EC%9E%AC%ED%82%B7%20%EC%9E%AC%ED%82%B7%20%EC%9E%AC%ED%82%B7%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%EB%82%A8%EC%84%B1%20%20%EA%B0%80%EB%94%94%EA%B1%B4%20%EC%9D%98%EB%A5%98%20%EB%82%A8%EC%84%B1&img=jkh HTTP/1.1" 200 89


 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능